# Solving IBM's Quantum Open Science Prize

This notebook serves as the main submission. The code inside this notebook is self-contained. But the approach builds upon the preparational work done in the previous notebooks. 


## 1. Initialize the environment

This approach uses

- a noise-free `QasmSimulator`
- the `Jakarta` quantum computer


In [1]:
# IBM account 
from qiskit.providers.aer import QasmSimulator

# load IBMQ Account data
from qiskit import IBMQ

# Noiseless simulated backend
sim = QasmSimulator()

# replace TOKEN with your API token string (https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq)
IBMQ.save_account("TOKEN", overwrite=True) 
account = IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')
jakarta = provider.get_backend('ibmq_jakarta')


## 2. Generate circuits

We put the creation of the trotterization circuit into a helper function `get_circuit`. The core logic of the circuit was taken from the provided documentation.

This allows us to create variants of the trotterization circuit:

- classically simulatable (short with few qubits) circuits
- the full trotterized simulation

Further, the function allows us to customize the number of trotter steps


In [2]:
# Trotterized circuit from IBM material
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

def get_circuit(measure, trotter_steps, X=True, Y=True, Z=True):
    """Create a trotterization circuit

    Args:
        measure (bool): Whether to include a measurement into the circuit
        trotter_steps (int): Number of trotter steps in the simulation
        X (bool): Whether to include the XX(t) gate into the circuit
        Y (bool): Whether to include the YY(t) gate into the circuit
        Z (bool): Whether to include the ZZ(t) gate into the circuit

    Returns:
        The Qiskit circuit

    """

    # YOUR TROTTERIZATION GOES HERE -- START (beginning of example)

    # Parameterize variable t to be evaluated at t=pi later
    t = Parameter('t')

    # Build a subcircuit for XX(t) two-qubit gate
    XX_qr = QuantumRegister(2)
    XX_qc = QuantumCircuit(XX_qr, name='XX')

    XX_qc.ry(np.pi/2,[0,1])
    XX_qc.cnot(0,1)
    XX_qc.rz(2 * t, 1)
    XX_qc.cnot(0,1)
    XX_qc.ry(-np.pi/2,[0,1])

    # Convert custom quantum circuit into a gate
    XX = XX_qc.to_instruction()

    # Build a subcircuit for YY(t) two-qubit gate
    YY_qr = QuantumRegister(2)
    YY_qc = QuantumCircuit(YY_qr, name='YY')

    YY_qc.rx(np.pi/2,[0,1])
    YY_qc.cnot(0,1)
    YY_qc.rz(2 * t, 1)
    YY_qc.cnot(0,1)
    YY_qc.rx(-np.pi/2,[0,1])

    # Convert custom quantum circuit into a gate
    YY = YY_qc.to_instruction()

    # Build a subcircuit for ZZ(t) two-qubit gate
    ZZ_qr = QuantumRegister(2)
    ZZ_qc = QuantumCircuit(ZZ_qr, name='ZZ')

    ZZ_qc.cnot(0,1)
    ZZ_qc.rz(2 * t, 1)
    ZZ_qc.cnot(0,1)

    # Convert custom quantum circuit into a gate
    ZZ = ZZ_qc.to_instruction()

    # Combine subcircuits into a single multiqubit gate representing a single trotter step
    num_qubits = 3

    Trot_qr = QuantumRegister(num_qubits)
    Trot_qc = QuantumCircuit(Trot_qr, name='Trot')

    for i in range(0, num_qubits - 1):
        if Z:
            Trot_qc.append(ZZ, [Trot_qr[i], Trot_qr[i+1]])
        
        if Y:
            Trot_qc.append(YY, [Trot_qr[i], Trot_qr[i+1]])
        
        if X:
            Trot_qc.append(XX, [Trot_qr[i], Trot_qr[i+1]])

    # Convert custom quantum circuit into a gate
    Trot_gate = Trot_qc.to_instruction()
    
    # YOUR TROTTERIZATION GOES HERE -- FINISH (end of example)


    # The final time of the state evolution
    target_time = np.pi

    # Number of trotter steps
    #trotter_steps = 8  ### CAN BE >= 4

    # Initialize quantum circuit for 3 qubits
    qr = QuantumRegister(7)
    cr = ClassicalRegister(3)
    qc = QuantumCircuit(qr, cr) if measure is True else QuantumCircuit(qr)

    # Prepare initial state (remember we are only evolving 3 of the 7 qubits on jakarta qubits (q_5, q_3, q_1) corresponding to the state |110>)
    qc.x([3,5])  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)

    # Simulate time evolution under H_heis3 Hamiltonian
    for _ in range(trotter_steps):
        qc.append(Trot_gate, [qr[1], qr[3], qr[5]])
        if not X or not Y or not Z:
            break
    
        

    # Evaluate simulation at target_time (t=pi) meaning each trotter step evolves pi/trotter_steps in time
    qc = qc.bind_parameters({t: target_time/trotter_steps})

    if measure:
        qc.measure([1,3,5], cr)

    
    return qc

## 3.Qiskit Measurement Result Wrapper

Our solution post-processes the measurements. Therefore, we overwrite the counts that we feed into the state tomography. Since the Qiskit `Result` object does not support overwriting the counts, we create a wrapper class.


In [3]:
# Import OwnResult from POST 12

from qiskit.result import Result

from qiskit.circuit.quantumcircuit import QuantumCircuit
from qiskit.exceptions import QiskitError
from qiskit.result.counts import Counts

class OwnResult(Result):
    
    def __init__(self, result):
        self._result = result
        self._counts = {}
            
        
    def get_counts(self, experiment=None):

        if experiment is None:
            exp_keys = range(len(self._result.results))
        else:
            exp_keys = [experiment]
        

        dict_list = []
        for key in exp_keys:
            exp = self._result._get_experiment(key)
            try:
                header = exp.header.to_dict()
            except (AttributeError, QiskitError):  # header is not available
                header = None

            if "counts" in self._result.data(key).keys():
                if header:
                    counts_header = {
                        k: v
                        for k, v in header.items()
                        if k in {"time_taken", "creg_sizes", "memory_slots"}
                    }
                else:
                    counts_header = {}
                    
                    
                # CUSTOM CODE STARTS HERE #######################
                dict_list.append(Counts(
                    self._counts[str(key)] if str(key) in map(lambda k: str(k), self._counts.keys()) else self._result.data(key)["counts"]
                    , **counts_header))
                # CUSTOM CODE ENDS HERE #######################
                
            elif "statevector" in self._result.data(key).keys():
                vec = postprocess.format_statevector(self._result.data(key)["statevector"])
                dict_list.append(statevector.Statevector(vec).probabilities_dict(decimals=15))
            else:
                raise QiskitError(f'No counts for experiment "{repr(key)}"')

        # Return first item of dict_list if size is 1
        if len(dict_list) == 1:
            return dict_list[0]
        else:
            return dict_list
        
        
    def set_counts(self, counts, experiment=None):
        self._counts[str(experiment) if experiment is not None else "0"] = counts

## Piece 4: State Tomography Mitigation

The `state_tomo` function performs the post processing of the simulation.


In [4]:
from qiskit.opflow import Zero, One
from qiskit.ignis.verification.tomography import StateTomographyFitter
from qiskit.quantum_info import state_fidelity

# Compute the state tomography based on the st_qcs quantum circuits and the results from those ciricuits
def state_tomo(result, st_qcs, modifiers, mitigate=False):
    """Calculate the fidelity of 

    Args:
        result (Object): The result of the circuit execution
        st_qcs (list[Circuit]): The 27 state tomography circuits
        modifiers (List[List[[float]]): A list of floats that modify the counts. One modifier per each state and each state tomography circuit
        mitigate (bool): Whether to mitigate the counts

    Returns:
        The fidelity of the quantum state

    """

    
    # The expected final state; necessary to determine state tomography fidelity
    target_state = (One^One^Zero).to_matrix()  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)
    
    own_res = OwnResult(result)
    
    idx = 0
    
    if mitigate:
        for experiment in st_qcs:
            exp_keys = [experiment]
            for key in exp_keys:

                exp = result._get_experiment(key)                
                counts = sorted_counts(result.get_counts(key))
                mitigated = {item[0]: item[1]*modifiers[idx][i] for i, item in enumerate(counts.items())}
                
                #print("original: ", sorted_counts(result.get_counts(key)))
                #print("mitigated: ", sorted_counts(mitigated))
                #print("\n")

                own_res.set_counts(mitigated, key)
    
            idx = idx + 1 
        
    
    # Fit state tomography results
    tomo_fitter = StateTomographyFitter(own_res if mitigate else result, st_qcs)
    rho_fit = tomo_fitter.fit(method='lstsq')
    # Compute fidelity
    fid = state_fidelity(rho_fit, target_state)
    return fid



/tmp/ipykernel_374/2168099722.py:2: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.verification.tomography import StateTomographyFitter


## 5: Create the training circuits

Our approach builds upon classically simulatable circuits. These are circuits that consist of a single trotter-gate and two of the three subcircuits. For example, a circuit with a trotter-gate that contains the XX and the YY gate.

`trotters` specifies the number of trotter steps that the overall simulation contains. 

In [6]:
# prepare the circuits
from qiskit.ignis.verification.tomography import state_tomography_circuits

# trotter steps
trotters = 12


train_st_qcs_xy = state_tomography_circuits(get_circuit(False, trotters, True, True, False), [1,3,5])
train_st_qcs_yz = state_tomography_circuits(get_circuit(False, trotters, True, True, False), [1,3,5])
train_st_qcs_zx = state_tomography_circuits(get_circuit(False, trotters, True, True, False), [1,3,5])



The `start_train_jobs` submits a quantum circuit to the Jakarta backend.

In [59]:
def start_train_jobs(qc, shots = 1024):
    t_qc = transpile(qc, jakarta)
    qobj = assemble(t_qc)
    jobs = jakarta.run(qobj, shots=shots)
    return jobs


The `reps` denote the number of times we need to run each subcircuit. The underlying rationale is that each subcircuit is contains two-thirds of one trotter gate. With three subcircuits, we have to repeat them `trotter/2` times to reach the same overall length.

We start each training circuit the respective number of times and obtain their job identifiers.

In [63]:
# repititions
reps = int(trotters/2)

# The XY transition
xy_jobs = list(map(lambda x: start_train_jobs(train_st_qcs_xy), range(reps)))

for job in xy_jobs:
    print("'{}',".format(job.job_id()))

/tmp/ipykernel_539/897617969.py:4: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  jobs = jakarta.run(qobj, shots=shots)


'623dcb95e32b42e943ecddd9',
'623dcb9709995c0174492dfd',
'623dcb9a538eba5f7361191c',
'623dcb9c0af65d852cd938ab',
'623dcb9ed97bff0994694bf7',
'623dcba00af65df04cd938ac',


In [64]:
# The YZ transition
yz_jobs = list(map(lambda x: start_train_jobs(train_st_qcs_yz), range(reps)))

for job in yz_jobs:
    print("'{}',".format(job.job_id()))

'623dcbbc74de0ed3c785b3a9',
'623dcbbee32b4255eeecddda',
'623dcbc10af65d824ad938ad',
'623dcbc374de0e195885b3aa',
'623dcbc5ecc41302c2b71c4e',
'623dcbc70af65d56fdd938ae',


In [65]:
# The ZX transition
zx_jobs = list(map(lambda x: start_train_jobs(train_st_qcs_zx), range(reps)))

for job in zx_jobs:
    print("'{}',".format(job.job_id()))

'623dcbdb19e6895d16c8128b',
'623dcbdd538eba0a5e61191e',
'623dcbdf74de0e1e7085b3ac',
'623dcbe1ecc413fc86b71c50',
'623dcbe3d97bff296f694bf8',
'623dcbe58293e956301e61e5',


## 6. Run the evaluation circuits

To evaluate the performance of the approach, we need a series of the complete state tomography circuits.

In [67]:
st_qcs = state_tomography_circuits(get_circuit(False, trotters, True, True, True), [1,3,5])
eval_jobs = list(map(lambda x: start_train_jobs(st_qcs), range(8)))

for job in eval_jobs:
    print("'{}',".format(job.job_id()))

'623dcc930af65d0467d938b1',
'623dcc9d74de0e600b85b3ad',
'623dcca7a2f72d26a6daba39',
'623dccb574de0e848485b3ae',
'623dccbf538eba278c611921',
'623dccc98293e907301e61e7',
'623dccd209995c47aa492dfe',
'623dccdbe32b425652ecdddc',


The following lists contain the job identifiers we obtained above.

In [7]:
job_ids_xy = [
    '623dcb95e32b42e943ecddd9',
    '623dcb9709995c0174492dfd',
    '623dcb9a538eba5f7361191c',
    '623dcb9c0af65d852cd938ab',
    '623dcb9ed97bff0994694bf7',
    '623dcba00af65df04cd938ac'
]

In [8]:
job_ids_yz = [
    '623dcbbc74de0ed3c785b3a9',
    '623dcbbee32b4255eeecddda',
    '623dcbc10af65d824ad938ad',
    '623dcbc374de0e195885b3aa',
    '623dcbc5ecc41302c2b71c4e',
    '623dcbc70af65d56fdd938ae'
]

In [9]:
job_ids_zx = [
    '623dcbdb19e6895d16c8128b',
    '623dcbdd538eba0a5e61191e',
    '623dcbdf74de0e1e7085b3ac',
    '623dcbe1ecc413fc86b71c50',
    '623dcbe3d97bff296f694bf8',
    '623dcbe58293e956301e61e5'
]

In [17]:
complete_job_ids = [
    '623d8d50a2f72db120dab8b0',
    '623d8d598293e978251e6068',
    '623d8d7209995c6e6d492c5c',
    '623d8d8cd97bff4aa9694a72',
    '623d8d9519e6897773c810fb',
    '623d8d9e19e6891489c810fc',
    '623d8da8e32b425c46ecdc7d',
    '623d8db219e689e45cc810ff'
]

## 7. Retrieve the completed results

Once the backend processed all the jobs, we can proceed. Therefore, we retrieve the jobs using the identifiers.


In [10]:
jobs_xy = list(map(jakarta.retrieve_job, job_ids_xy))
jobs_yz = list(map(jakarta.retrieve_job, job_ids_yz))
jobs_zx = list(map(jakarta.retrieve_job, job_ids_zx))

## 8. Calculate the modifiers

At the core of our approach are the count modifiers. Each modifier is the ratio between the noiseless and the noisy count of one training circuit. For instance, if the noiseless count of a training circuit is 50 and the noisy is 25, then the modifier is 2.


In [11]:
from collections import Counter
from functools import reduce
from qiskit import assemble, execute, transpile


def sorted_counts(counts):
    complete = dict(reduce(lambda a, b: a.update(b) or a, [{'000': 0, '001': 0, '010': 0, '011': 0, '100': 0, '101': 0, '110': 0, '111': 0}, counts], Counter()))
    return {k: v for k, v in sorted(complete.items(), key=lambda item: item[0])}


def get_jakarta_modifiers(qc, job, shots = 1024, display=True):
    
    t_qc_sim = transpile(qc, sim)
    noiseless_result = sim.run( assemble(t_qc_sim), shots=shots).result()
    noiseless_counts = sorted_counts(noiseless_result.get_counts())
    
    #t_qc = transpile(qc, sim_noisy_jakarta)
    #qobj = assemble(t_qc)
    counts = sorted_counts(job.result().get_counts(qc))
    
    zipped = list(zip(noiseless_counts.values(), counts.values()))
    modifier = list(map(lambda pair: pair[0]/pair[1] if pair[1] > 0 else 1, zipped))

    if display is True:
        print("noisy:     ", counts)
        print("nose-free: ", noiseless_counts)

        print("modifier: ", modifier)
        print("\n")
    
    return modifier



Further, we reorder the data to get a list of 27 (one per state tomography circuit). Each item contains 8 modifiers. This is one per possible state (from `000` to `111`).

In [39]:
from functools import reduce

# each variable is a list of the trotters/2=6 jobs. Each job is a list of 27 state tomography circuits. Each of these lists 
list_modifiers_xy = list(map(lambda job: list(map(lambda qc: get_jakarta_modifiers(qc,job, display=False), train_st_qcs_xy)), jobs_xy))
list_modifiers_yz = list(map(lambda job: list(map(lambda qc: get_jakarta_modifiers(qc, job, display=False), train_st_qcs_yz)), jobs_yz))
list_modifiers_zx = list(map(lambda job: list(map(lambda qc: get_jakarta_modifiers(qc, job, display=False), train_st_qcs_zx)), jobs_zx))


# reorder the data
state_tomography_xy = list(zip(*list_modifiers_xy))
state_tomography_yz = list(zip(*list_modifiers_yz))
state_tomography_zx = list(zip(*list_modifiers_zx))

zipped = list(map(lambda l: [item for sublist in l  for item in sublist], list(zip(state_tomography_xy, state_tomography_yz, state_tomography_zx))))


def calc_modifier(stqc):
    mods = list(map(lambda mod_list: reduce(lambda a,b: a*b, mod_list, 1), zip(*stqc)))
    return mods
    

mods = list(map(calc_modifier,zipped))

## 9. Evaluate the performance

The following step calculates the fidelity of each evaluation job.


In [41]:

shots = 1024
st_qcs = state_tomography_circuits(get_circuit(False, trotters, True, True, True), [1,3,5])

noisefree_job = execute(st_qcs, sim, shots=shots)
noisefree_fid = state_tomo(noisefree_job.result(), st_qcs, mods, mitigate=False)

# Compute tomography fidelities for each repetition
noisy_fids = []
mitigated_fids = []

for jobid in complete_job_ids:
    noisy_job = jakarta.retrieve_job(jobid)
    
    noisy_fid = state_tomo(noisy_job.result(), st_qcs, mods, mitigate=False)
    mitigated_fid = state_tomo(noisy_job.result(), st_qcs, mods, mitigate=True)

    noisy_fids.append(noisy_fid)
    mitigated_fids.append(mitigated_fid)



print('noisy state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean(noisy_fids), np.std(noisy_fids)))
print('noise-free state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean([noisefree_fid]), np.std([noisefree_fid])))
print('mitigated state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean(mitigated_fids), np.std(mitigated_fids)))


noisy state tomography fidelity = 0.1109 ± 0.0036
noise-free state tomography fidelity = 0.9625 ± 0.0000
mitigated state tomography fidelity = 0.5097 ± 0.0091


In [ ]:
We calculate further statistics about the error reduction.

In [43]:
unmitigated = np.mean(noisy_fids)
ideal = np.mean([noisefree_fid])
mitigated = np.mean(mitigated_fids)

error_unmitigated = abs(unmitigated-ideal)
error_mitigated = abs(mitigated-ideal)

print("Error (unmitigated):", error_unmitigated)
print("Error (mitigated):", error_mitigated)

print("Relative error (unmitigated):", (error_unmitigated/ideal))
print("Relative error (mitigatedR):", error_mitigated/ideal)

print(f"Error reduction: {(error_unmitigated-error_mitigated)/error_unmitigated :.1%}.")

Error (unmitigated): 0.8515999368634573
Error (mitigated): 0.4527662343064023
Relative error (unmitigated): 0.8848172148711483
Relative error (mitigatedR): 0.4704267122214712
Error reduction: 46.8%.


## 10. Assessment

We see a mitigated state tomography fidelity of **0.5097** based on 12 trotter steps. With a noisefree fidelity of 0.96 and an unmitigated fidelity of 0.11, we achieve an overall error reduction of 46.8%.
